In [1]:
import pandas as pd

from jobsfinder.gpts import has_sales_roles
from jobsfinder.core import limit_parallel

In [2]:
df = pd.read_csv("../data/04_jobs.csv")

In [3]:
df.status.value_counts()

status
No jobs              1471
Loop detected         226
Job open apply         56
Job list               43
Max depth reached       6
Name: count, dtype: int64

In [4]:
hiring = df[df.status == "Job list"].copy()

In [5]:
len(hiring)

43

In [6]:
hiring.head(2)

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website,history,status,error,jobs
105,Mintlify,http://mintlify.com,Success,Success,Mintlify \- The modern standard for documentat...,114033,6819,valid,"[""http://mintlify.com"", ""/careers""]",Job list,NaN,"[""Founding Account Executive"", ""Senior Fronten..."
117,Zuplo,https://zuplo.com,Success,Success,Zuplo API ManagementProduct[Docs](https://zupl...,363987,8175,valid,"[""https://zuplo.com"", ""/careers""]",Job list,NaN,"[""Designer Extraordinaire""]"


In [7]:
await has_sales_roles(hiring.jobs.iloc[0])

SalesRoles(qualified=True, best_roles=['Founding Account Executive', 'Head of Growth'], email_line="I saw you're hiring for a founding account executive and a head of growth.")

In [8]:
tasks = [has_sales_roles(x) for x in qualified.jobs]

quals = await limit_parallel(tasks)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43/43 [01:51<00:00,  2.60s/it]


In [11]:
hiring['qualified'] = [x.qualified for x in quals]
hiring['best_roles'] = [",".join(x.best_roles) for x in quals]
hiring['line'] = [x.email_line for x in quals]

In [12]:
hiring.qualified.value_counts()

qualified
False    33
True     10
Name: count, dtype: int64

In [20]:
qualified = hiring[hiring.qualified].copy()

In [21]:
len(qualified)

10

In [22]:
qualified.Website

105                       http://mintlify.com
184                    http://edgeandnode.com
249                 https://www.paraform.com/
312                   https://codecrafters.io
700                   https://bloomsystems.io
732     https://www.climatetechcocktails.com/
1007                https://mobilityroute.com
1454            https://www.mycourseflow.com/
1514                http://www.outamation.com
1770               https://www.tensordock.com
Name: Website, dtype: object

checking manually, 2 (precision) errors in the whole flow

In [23]:
def _create_email(x):
    return f"""
Hey [firstName],

{x}. Businesss must be going good!

Given your trajectory, I assume your doing everything to increase conversion. That's also the reason we've built UserGems. You can automatically capture buying signals from your prospect & put your pipeline on auto-pilot.

Let me know if that's of interest.

Best,
[Your Name]
    """.strip()

In [24]:
qualified['email'] = qualified.line.apply(_create_email)

In [26]:
qualified.to_csv("../data/06_qualifid.csv", index=False)